In [ ]:
import os
import cv2
import torch
import traceback
from ultralytics import YOLO

In [ ]:


# --- --- 配置參數 --- ---

# 數據集設定檔路徑
DATASET_YAML_PATH = 'Capoo-Dataset/data.yaml' # 請確保這個路徑指向您的 data.yaml

# 模型相關
BASE_MODEL = 'yolov8n.pt'      # 使用預訓練模型開始 ('yolov8n.pt', 'yolov8s.pt', ...)
# BASE_MODEL = 'yolov8n.yaml' # 或者從 YAML 結構檔開始訓練 (通常不推薦，除非數據量很大)

# 訓練參數
EPOCHS = 100                   # 訓練世代數
IMG_SIZE = 640                 # 圖片尺寸
BATCH_SIZE = 8                 # 批次大小 (依 GPU VRAM 調整，顯存不足請減小)
PATIENCE = 20                  # Early stopping 的耐心值 (多少個 epoch 沒有改善就停止)
TRAINING_NAME = 'capoo_yolov8_run_combined' # 本次訓練的名稱 (結果儲存資料夾)
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu' # 自動選擇 GPU 或 CPU
WORKERS = 4                    # 資料載入執行緒數 (依 CPU 核心數調整)

# 推論(Inference)設定
# 訓練完成後用來測試最佳模型的圖片或影片路徑
# *** 請務必將此路徑修改為您想用來測試的實際圖片或影片路徑 ***
INFERENCE_SOURCE = 'path/to/your/test_image.jpg' # 例如: 'Capoo-Dataset/test/images/capoo_test_01.jpg'
# INFERENCE_SOURCE = 'path/to/your/test_video.mp4'

# --- --- 訓練函數 --- ---

def train_capoo_model(data_yaml, base_model, epochs, imgsz, batch, patience, name, device, workers):
    """訓練 YOLOv8 模型"""
    model = YOLO(base_model)
    
    print("\n" + "="*30)
    print(" 開始模型訓練 ".center(30, "="))
    print("="*30)
    print(f"數據集設定檔: {data_yaml}")
    print(f"基礎模型: {base_model}")
    print(f"訓練世代數 (Epochs): {epochs}")
    print(f"圖片尺寸 (imgsz): {imgsz}")
    print(f"批次大小 (Batch): {batch}")
    print(f"Early Stopping 耐心值 (Patience): {patience}")
    print(f"訓練設備 (Device): {device}")
    print(f"資料載入執行緒數 (Workers): {workers}")
    print(f"訓練名稱 (Name): {name}")
    print("="*30 + "\n")

    best_model_path = None
    try:
        results = model.train(
            data=data_yaml,
            epochs=epochs,
            imgsz=imgsz,
            batch=batch,
            patience=patience,
            name=name,
            device=device,
            workers=workers,
            project='runs/detect', # 指定 runs 的根目錄 (預設就是 runs/detect)
            exist_ok=False,        # 如果實驗名稱已存在，不要覆蓋，避免意外 (設 True 可覆蓋)
            # 其他可選參數...
            # val=True,            # 預設即為 True，每個 epoch 後進行驗證
            # plots=True           # 預設即為 True，產生訓練曲線圖等
        )

        # 從 results 物件獲取最佳模型路徑 (適用於較新版本的 ultralytics)
        # 通常位於 results.save_dir / 'weights' / 'best.pt'
        save_dir = results.save_dir
        best_model_path_check = os.path.join(save_dir, 'weights', 'best.pt')

        if os.path.exists(best_model_path_check):
             best_model_path = best_model_path_check
             print("\n" + "="*30)
             print(" 訓練成功完成 ".center(30, "="))
             print(f"訓練結果儲存在: {save_dir}")
             print(f"最佳模型權重: {best_model_path}")
             print("="*30 + "\n")
        else:
             print("\n" + "="*30)
             print(" 訓練完成，但未找到 best.pt ".center(30, "="))
             print(f"請檢查儲存目錄: {save_dir}")
             print("="*30 + "\n")
             # Fallback or further checks needed

    except Exception as e:
        print(f"\n!!! 訓練過程中發生錯誤 !!!")
        print(f"錯誤訊息: {e}")
        traceback.print_exc()
        print("-" * 30)

    return best_model_path

# --- --- 推論函數 --- ---

def run_capoo_inference(model_path, source):
    """使用訓練好的模型進行推論"""
    if model_path is None:
        print("錯誤：未提供模型路徑，無法進行推論。")
        return
    if not os.path.exists(model_path):
        print(f"錯誤：找不到模型檔案 {model_path}")
        return
    if not os.path.exists(source):
        print(f"錯誤：找不到推論來源檔案/路徑 {source}")
        return

    print("\n" + "="*30)
    print(" 開始使用自訂模型進行推論 ".center(30, "="))
    print(f"使用模型: {model_path}")
    print(f"推論來源: {source}")
    print("="*30 + "\n")

    # 載入您訓練好的自訂模型
    model = YOLO(model_path)

    # 執行推論
    results = model(source, stream=True) # stream=True 適用於影片或大量圖片，節省記憶體

    output_filename = "inference_output.jpg"
    output_video_filename = "inference_output.mp4"
    is_video = source.lower().endswith(('.mp4', '.avi', '.mov', '.mkv'))
    
    if is_video:
        print("處理影片中... 按 'q' 鍵停止。")
        # 獲取影片的基本信息以創建 VideoWriter
        cap = cv2.VideoCapture(source)
        width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
        fps = cap.get(cv2.CAP_PROP_FPS)
        cap.release() # 釋放捕獲

        # 創建 VideoWriter 物件
        fourcc = cv2.VideoWriter_fourcc(*'mp4v') # 或者使用 'XVID'
        out_video = cv2.VideoWriter(output_video_filename, fourcc, fps, (width, height))
        
        frame_count = 0
        for result in results:
            frame_annotated = result.plot() # 獲取標註後的幀
            cv2.imshow("Custom Capoo Inference (Press 'q' to quit)", frame_annotated)
            out_video.write(frame_annotated) # 寫入影片檔
            frame_count += 1

            if cv2.waitKey(1) & 0xFF == ord('q'): # 按 q 鍵退出
                break
        
        out_video.release() # 釋放 VideoWriter
        cv2.destroyAllWindows()
        if frame_count > 0:
             print(f"\n推論完成！已處理 {frame_count} 幀。")
             print(f"標註後的影片已儲存為: {output_video_filename}")
        else:
             print(f"\n未能處理任何影片幀，請檢查影片來源 {source}")

    else: # 處理單張圖片
        print("處理圖片中...")
        for result in results:
            img_annotated = result.plot()
            cv2.imshow("Custom Capoo Inference", img_annotated)
            print("按任意鍵關閉圖片...")
            cv2.waitKey(0)
            cv2.destroyAllWindows()
            
            # 儲存標註後的圖片
            cv2.imwrite(output_filename, img_annotated)
            print(f"推論完成！標註後的圖片已儲存為: {output_filename}")
            break # 單張圖片處理完就退出迴圈

# --- --- 主執行區塊 --- ---

if __name__ == '__main__':
    # 0. 檢查環境
    print("--- 環境檢查 ---")
    print(f"Python version: {os.sys.version.split()[0]}")
    print(f"PyTorch version: {torch.__version__}")
    print(f"CUDA available: {torch.cuda.is_available()}")
    if torch.cuda.is_available():
        print(f"CUDA version: {torch.version.cuda}")
        print(f"Detected GPU: {torch.cuda.get_device_name(torch.cuda.current_device())}")
    print("---------------")

    # 1. 執行訓練
    #    如果訓練成功，會返回最佳模型的路徑
    best_model_file = train_capoo_model(
        data_yaml=DATASET_YAML_PATH,
        base_model=BASE_MODEL,
        epochs=EPOCHS,
        imgsz=IMG_SIZE,
        batch=BATCH_SIZE,
        patience=PATIENCE,
        name=TRAINING_NAME,
        device=DEVICE,
        workers=WORKERS
    )

    # 2. 如果訓練成功，使用最佳模型進行推論
    if best_model_file:
        run_capoo_inference(model_path=best_model_file, source=INFERENCE_SOURCE)
    else:
        print("\n由於訓練未成功或未找到最佳模型，跳過推論步驟。")

    print("\n腳本執行完畢。")